In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd
import pyrex
#import pyrex.custom.ara as ara
from pyrex.internal_functions import normalize
import os
import sys
import seaborn as sns
import polReco_util as util
import scipy

import ROOT
from ROOT import TCanvas, TGraph
from ROOT import gROOT
from ROOT import gInterpreter, gSystem
from ROOT import TChain, TSelector, TTree
from scipy.stats import norm
from scipy.stats import rayleigh

#add headers from AraSim. Not sure if all of them are needed, and I'm lazy to check that. MAK SURE to change the location of the headers
# gInterpreter.ProcessLine('#include "/cvmfs/ara.opensciencegrid.org/trunk/centos7/source/AraSim/Position.h"')
# gInterpreter.ProcessLine('#include "/cvmfs/ara.opensciencegrid.org/trunk/centos7/source/AraSim/Report.h"')
# gInterpreter.ProcessLine('#include "/cvmfs/ara.opensciencegrid.org/trunk/centos7/source/AraSim/Detector.h"')
# gInterpreter.ProcessLine('#include "/cvmfs/ara.opensciencegrid.org/trunk/centos7/source/AraSim/Settings.h"')
# gInterpreter.ProcessLine('#include "/cvmfs/ara.opensciencegrid.org/trunk/centos7/source/AraSim/EarthModel.h"')
# gInterpreter.ProcessLine('#include "/cvmfs/ara.opensciencegrid.org/trunk/centos7/source/AraSim/Event.h"')
# gInterpreter.ProcessLine('#include "/cvmfs/ara.opensciencegrid.org/trunk/centos7/source/AraSim/Spectra.h"')
# gInterpreter.ProcessLine('#include "/cvmfs/ara.opensciencegrid.org/trunk/centos7/source/AraSim/Interaction.h"')


gInterpreter.ProcessLine('#include "/cvmfs/ara.opensciencegrid.org/trunk/centos7/source/libRootFftwWrapper/include/FFTtools.h"')


# gSystem.Load('/cvmfs/ara.opensciencegrid.org/trunk/centos7/source/AraSim/libAra.so') #load the simulation event library. You might get an error asking for the eventSim dictionry. To solve that, go to where you compiled AraSim, find that file, and copy it to where you set LD_LIBRARY_PATH.
# gSystem.Load('/cvmfs/ara.opensciencegrid.org/trunk/centos7/ara_build/lib/libAraEvent.so')
gSystem.Load('/users/PAS0654/jflaherty13/source/AraRoot/AraRoot_build/lib/libAraEvent.so')
gSystem.Load("/cvmfs/ara.opensciencegrid.org/trunk/centos7/source/libRootFftwWrapper/build/libRootFftwWrapper.so")

gInterpreter.ProcessLine('#include "/users/PAS0654/jflaherty13/source/AraSim/Position.h"')
gInterpreter.ProcessLine('#include "/users/PAS0654/jflaherty13/source/AraSim/Report.h"')
gInterpreter.ProcessLine('#include "/users/PAS0654/jflaherty13/source/AraSim/Detector.h"')
gInterpreter.ProcessLine('#include "/users/PAS0654/jflaherty13/source/AraSim/Settings.h"')
gInterpreter.ProcessLine('#include "/users/PAS0654/jflaherty13/source/AraSim/IceModel.h"')
gInterpreter.ProcessLine('#include "/users/PAS0654/jflaherty13/source/AraSim/EarthModel.h"')
gInterpreter.ProcessLine('#include "/users/PAS0654/jflaherty13/source/AraSim/Event.h"')
gInterpreter.ProcessLine('#include "/users/PAS0654/jflaherty13/source/AraSim/Spectra.h"')
# gInterpreter.ProcessLine('#include "/users/PAS0654/jflaherty13/source/AraSim/Interaction.h"')

# sys.path.append("/cvmfs/ara.opensciencegrid.org/trunk/centos7/source/ARA_cvmfs/root_build/lib/") # go to parent dir
#sys.path.append("/users/PCON0003/cond0068/.local/lib/python3.7/site-packages/")  #I don't have access to this directory in Jorge's account
#sys.path.append("/users/PAS0654/jflaherty13/.local/lib/python3.9/site-packages/")
# sys.path.append("/users/PCON0003/cond0068/pyrex_sims/fromBen/thesis_work/pyrex-custom/analysis/custom/analysis/")
# import ROOT
# import math
# from ROOT import TH1D,TF1, gRandom, gPad, gStyle
#import matplotlib as mpl
#import matplotlib.pyplot as plt
# from ROOT import TChain, TSelector, TTree
#import itertools

# mpl.use('agg') 
mpl.rcParams['text.usetex'] = True
#mpl.rcParams['text.latex.preamble'] = [r'\usepackage{amsmath}'] #for \text command
#mpl.rcParams['text.latex.unicode'] = True
mpl.rcParams['mathtext.rm'] = 'Times New Roman'
mpl.rcParams['mathtext.it'] = 'Times New Roman:italic'
mpl.rcParams['mathtext.bf'] = 'Times New Roman:bold'

mpl.rc('font', family='serif', size=12)
mpl.rcParams['xtick.labelsize'] = 14
mpl.rcParams['ytick.labelsize'] = 14
mpl.rcParams['xtick.major.size'] = 5
mpl.rcParams['ytick.major.size'] = 5

mpl.rcParams['axes.titlesize'] = 18
mpl.rcParams['axes.labelsize'] = 18
mpl.rc('font', size=16)
mpl.rc('axes', titlesize=20)

# import mplcyberpunk
# plt.style.use("cyberpunk")

#current_palette = sns.color_palette('colorblind', 10)
current_palette = sns.color_palette('bright', 10)

In [ ]:
stationID = 2 #2
runNumber = 90 #12557
subsetNumber = '' #102
# recoSrcFolder = "/users/PAS0654/jflaherty13/source/AraSim/outputs/updatedIdlPulseModelNoNoise/interferometry/recangle_reco_out_run_"+str(runNumber)+".root"
# rawSrcFolder = "/users/PAS0654/jflaherty13/source/AraSim/outputs/updatedIdlPulseModelNoNoise/AraOut.setup_variablePsi.txt.run"+str(runNumber)+".root"


# recoSrcFolder = "/users/PAS0654/jflaherty13/source/AraSim/outputs/updatedIdlPulseModelHpolGain/interferometry/recangle_reco_out_run_"+str(runNumber)+".root"
# rawSrcFolder = "/users/PAS0654/jflaherty13/source/AraSim/outputs/updatedIdlPulseModelHpolGain/AraOut.setup_variablePsi.txt.run"+str(runNumber)+".root"

# recoSrcFolder = "/users/PAS0654/jflaherty13/source/AraSim/outputs/updatedIdlPulseModelHpolGainNoNoise/interferometry/recangle_reco_out_run_"+str(runNumber)+".root"
# rawSrcFolder = "/users/PAS0654/jflaherty13/source/AraSim/outputs/updatedIdlPulseModelHpolGainNoNoise/AraOut.setup_variablePsi="+str(runNumber)+".txt.run"+str(runNumber)+".root"

# recoSrcFolder = "/users/PAS0654/jflaherty13/source/AraSim/outputs/updatedIdlPulseModelHpolGain/interferometry/recangle_reco_out_run_"+str(runNumber)+".root"
# rawSrcFolder = "/users/PAS0654/jflaherty13/source/AraSim/outputs/updatedIdlPulseModelHpolGainNoNoise/AraOut.setup_variablePsi="+str(runNumber)+".txt.run"+str(runNumber)+".root"
# rawSrcFolder = "/users/PAS0654/jflaherty13/source/AraSim/outputs/updatedIdlPulseModelHpolGainNoNoiseNoResponse/AraOut.setup_variablePsi="+str(runNumber)+".txt.run"+str(runNumber)+".root"
# rawSrcFolder = "/users/PAS0654/jflaherty13/source/AraSim/outputs/updatedIdlPulseModelHpolGainNoResponse/AraOut.setup_variablePsi="+str(runNumber)+".txt.run"+str(runNumber)+".root"
# recoSrcFolder = "/users/PAS0654/jflaherty13/source/AraSim/outputs/updatedIdlPulseModelHpolGainNoNoiseNoResponse/interferometry/recangle_reco_out_run_"+str(runNumber)+".root"

# rawSrcFolder = "/users/PAS0654/jflaherty13/source/AraSim/outputs/test/AraOut.setup_variablePsi="+str(runNumber)+".txt.run"+str(runNumber)+".root"
# recoSrcFolder = "/users/PAS0654/jflaherty13/source/AraSim/outputs/test/interferometry/recangle_reco_out_run_"+str(runNumber)+".root"

rawSrcFolder = "/users/PAS0654/jflaherty13/source/AraSim/outputs/20230710_pulserSims_1000m/withNoise/AraOut.setup_variablePsi="+str(runNumber)+".txt.run"+str(runNumber)+".root"
recoSrcFolder = "/users/PAS0654/jflaherty13/source/AraSim/outputs/20230710_pulserSims_1000m/interferometry/recangle_reco_out_run_"+str(runNumber)+".root"
deconvolvedFilename = "/users/PAS0654/jflaherty13/source/AraSim/outputs/20230710_pulserSims_1000m/withNoise/deconvolution/deconvolvedWaveforms_run_"+str(runNumber)+".root"

# rawSrcFolder = "/users/PAS0654/jflaherty13/source/AraSim/outputs/20230710_pulserSims_1000m/noNoise/AraOut.setup_variablePsi="+str(runNumber)+".txt.run"+str(runNumber)+".root"
# recoSrcFolder = "/users/PAS0654/jflaherty13/source/AraSim/outputs/20230710_pulserSims_1000m/interferometry/recangle_reco_out_run_"+str(runNumber)+".root"
# deconvolvedFilename = "/users/PAS0654/jflaherty13/source/AraSim/outputs/20230710_pulserSims_1000m/noNoise/deconvolution/deconvolvedWaveforms_run_"+str(runNumber)+".root"


# outputFolder = "newPolRecoV38/A"+str(stationID)+"/"

dataTypeFlag = 1
signalTypeFlag = 0
noiseTypeFlag = 0
gainBalance = 0
deconvolution = False

rawFilename = rawSrcFolder
recoFilename = recoSrcFolder

In [ ]:
print("runNumber = " + str(runNumber))
print("Sourcing vertex reconstruction from " + str(recoSrcFolder))
print("Sourcing raw data from " + str(rawSrcFolder))

In [ ]:
# recoFilename = "/users/PAS0654/jflaherty13/source/AraSim/outputs/alisaIdlPulseModel/interferometry/recangle_reco_out_run_45.root"
recoInputFile = ROOT.TFile.Open(recoFilename)


In [ ]:
# rawFilename = "/users/PAS0654/jflaherty13/source/AraSim/outputs/alisaIdlPulseModel/AraOut.setup_variablePsi.txt.run45.root"
rawInputFile = ROOT.TFile.Open(rawFilename)

In [ ]:
if (dataTypeFlag == 0):
    rawInputFile = ROOT.TFile.Open(rawFilename)
    recoInputFile = ROOT.TFile.Open(recoFilename)

    eventTree = rawInputFile.Get("eventTree")
    vertexReco = recoInputFile.Get("vertexReco")

    rawEvent = ROOT.RawAtriStationEvent()
    eventTree.SetBranchAddress("event",ROOT.AddressOf(rawEvent))

    totalRawEvents = eventTree.GetEntries()
    print('total raw events:', totalRawEvents)

    totalRecoEvents = vertexReco.GetEntries()
    print('total reco events:', totalRecoEvents)
    
elif (dataTypeFlag==1):
    rawInputFile = ROOT.TFile.Open(rawFilename)
    recoInputFile = ROOT.TFile.Open(recoFilename)
    
    # eventTree = rawInputFile.Get("eventTree")
    # simTree = rawInputFile.Get("AraTree2")
    
    file_list = []
    file_list.append(rawFilename)
    
    eventTree = TChain("eventTree") #Define chain and tree that needs to be read. "VTree" in this case.
    simSettingsTree = TChain("AraTree")
    simTree = TChain("AraTree2")
    vertexReco = recoInputFile.Get("vertexReco")

    for line in file_list:
        eventTree.AddFile(line)
        simTree.AddFile(line)
        simSettingsTree.AddFile(line)
        
    reportPtr = ROOT.Report()#report pointer
    eventPtr = ROOT.Event()
    detectorPtr = ROOT.Detector()
    interactionPtr = ROOT.Interaction()
    stationPtr = ROOT.ARA_station()

    usefulEvent = ROOT.UsefulAtriStationEvent()
    rawEvent = ROOT.RawAtriStationEvent()
    eventTree.SetBranchAddress("UsefulAtriStationEvent",ROOT.AddressOf(usefulEvent))
    eventTree.SetBranchAddress("RawAtriStationEvent",ROOT.AddressOf(rawEvent))
    simTree.SetBranchAddress("report",ROOT.AddressOf(reportPtr))
    simTree.SetBranchAddress("event", ROOT.AddressOf(eventPtr))
    simTree.SetBranchAddress("Nu_Interaction", ROOT.AddressOf(interactionPtr))
    simSettingsTree.SetBranchAddress("detector", ROOT.AddressOf(detectorPtr))
    simSettingsTree.SetBranchAddress("stations", ROOT.AddressOf(stationPtr))
    simSettingsTree.GetEntry(0)

    totalRawEvents = eventTree.GetEntries()
    print('total raw events:', totalRawEvents) 
    
    totalRecoEvents = vertexReco.GetEntries()
    print('total reco events:', totalRecoEvents)
    
if (totalRecoEvents != totalRawEvents):
    print("ERROR: Raw event file and reconstructed event file have different number of events! Ensure that you are using compatible files!")
    sys.exit
    

In [ ]:
evt=0
vertexReco.GetEntry(evt)
eventTree.GetEntry(evt)
util.plotDeconvolvedWaveform(usefulEvent, vertexReco)

In [ ]:
util.plotDeconvolvedWaveform(usefulEvent, vertexReco, deconvolution=False)

In [ ]:
fig, axes = util.plotDeconvolvedWaveform(usefulEvent, vertexReco, deconvolution=False, channelPair=0, Hilbert=True)
# axes[0].set_ylim([-400,400])
# axes[1].set_ylim([-400,400])

In [ ]:
#Making example plot for ICRC paper showing different polarization angles.
primaryFont = 15
secondaryFont = 15
fig, axes = util.plotDeconvolvedWaveform(usefulEvent, vertexReco, deconvolution=False, channelPair=1)
fig.set_size_inches(6,8)
fig.suptitle("$\Psi$ = " + str(runNumber) + "º", fontsize=2*primaryFont)
fig.supylabel("Amplitude [arb]",fontsize = primaryFont, x=-0.05)
fig.supxlabel("Time [ns]",fontsize = primaryFont)
axes[0].set_title("VPol Channel", fontsize=secondaryFont)
axes[1].set_title("HPol Channel", fontsize=secondaryFont)

vMin=-1000
vMax=1000
for ax in axes:
    ax.set_ylim([vMin,vMax])

In [ ]:
#Making example plot for ICRC paper showing different polarization angles.
primaryFont = 15
secondaryFont = 15
fig, axes = util.plotDeconvolvedWaveform(usefulEvent, vertexReco, deconvolution=False, channelPair=2)
fig.set_size_inches(6,8)
fig.suptitle("$\Psi$ = " + str(runNumber), fontsize=2*primaryFont)
fig.supylabel("Amplitude [arb]",fontsize = primaryFont)
fig.supxlabel("Time [ns]",fontsize = primaryFont)
axes[0].set_title("VPol Channel", fontsize=secondaryFont)
axes[1].set_title("HPol Channel", fontsize=secondaryFont)
# for ax in axes:
#     ax.set_xticks(fontsize=secondaryFont)
#     ax.set_yticks(fontsize=secondaryFont)

In [ ]:

# fig, axes = util.plotDeconvolvedWaveform(usefulEvent, vertexReco, deconvolution=True, channelPair=2, Hilbert=True)
# axes[0].set_ylim([-450000,450000])
# axes[1].set_ylim([-450000,450000])

In [ ]:

# fig, axes = util.plotDeconvolvedWaveform(usefulEvent, vertexReco, deconvolution=True, channelPair=2, Hilbert=True, station=2, configuration=6)
# axes[0].set_ylim([-450000,450000])
# axes[1].set_ylim([-450000,450000])

In [ ]:
from pyrex.internal_functions import (normalize, complex_bilinear_interp, complex_interp)
import pyrex.custom.envelope_reco as reco
import pyrex.custom.ara as ara
from pyrex.internal_functions import (normalize, complex_bilinear_interp,
                                      complex_interp)
import scipy.signal as signal

In [ ]:
ch=0
theta = np.array(vertexReco.reco_arrivalThetas_out)[ch]
phi = np.array(vertexReco.reco_arrivalPhis_out)[ch]




voltage, time = util.extractChannelWaveform(usefulEvent, ch)
polarization=np.array([-np.sin(phi),np.cos(phi),-1/np.sin(theta)])
ant = ara.VpolAntenna(name="Dummy Vpol", position=(0, 0, 0), power_threshold=0)
sampRate = len(time)/(max(time)-min(time))
b,a = signal.bessel(4, [0.15,0.4], 'bandpass', analog=False, fs=sampRate)
fft_v, fft_f, dT = util.doFFT(time,voltage)
response_filter = np.array(ant.interpolate_filter(fft_f*1E6))
dir_res = ant.antenna.directional_response(theta=theta, phi=phi, polarization=polarization)(fft_f*1E6)
heff = ant.antenna.frequency_response(fft_f*1E6)
response_antenna = dir_res*heff
response = response_antenna
deDis_wf = np.divide(fft_v,abs(response))
response = np.divide(response,abs(response))
deDis_wf = np.divide(deDis_wf,response)  #What is the purpose of this step?  The waveform seems to break without it. - JCF 6/19/2023
deDis_wf = np.nan_to_num(deDis_wf)
revert = util.doInvFFT(deDis_wf)
deDis_wf = signal.lfilter(b, a, revert)

In [ ]:
ant = ara.VpolAntenna(name="Dummy Vpol", position=(0, 0, 0), power_threshold=0, station=2, channel=0, configuration=6)

In [ ]:
freq = np.array(range(0, 1000, 100))
freq

In [ ]:
ROOT.Detector().GetGain_1D_OutZero(500,90,90,0,0)

In [ ]:
ch=0
theta = np.array(vertexReco.reco_arrivalThetas_out)[ch]
phi = np.array(vertexReco.reco_arrivalPhis_out)[ch]



voltage, time = util.extractChannelWaveform(usefulEvent, ch)
polarization=np.array([-np.sin(phi),np.cos(phi),-1/np.sin(theta)])
ant = ara.VpolAntenna(name="Dummy Vpol", position=(0, 0, 0), power_threshold=0)
sampRate = len(time)/(max(time)-min(time))
b,a = signal.bessel(4, [0.15,0.4], 'bandpass', analog=False, fs=sampRate)
fft_v, fft_f, dT = util.doFFT(time,voltage)
response_filter = np.array(ant.interpolate_filter(fft_f*1E6))
dir_res = ant.antenna.directional_response(theta=theta, phi=phi, polarization=polarization)(fft_f*1E6)
heff = ant.antenna.frequency_response(fft_f*1E6)
response_antenna = dir_res*heff
response = response_antenna
deDis_wf = np.divide(fft_v,abs(response))
response = np.divide(response,abs(response))
deDis_wf = np.divide(deDis_wf,response)  #What is the purpose of this step?  The waveform seems to break without it. - JCF 6/19/2023
deDis_wf = np.nan_to_num(deDis_wf)
revert = util.doInvFFT(deDis_wf)
deDis_wf = signal.lfilter(b, a, revert)

# gain = np.empty(fft_f.shape)
# for i in range(5):
#     gain[i] = ROOT.Detector().GetGain_1D_OutZero(fft_f[i],theta,phi,ch)

In [ ]:
plt.plot(time,voltage)

In [ ]:
plt.plot(time,deDis_wf)

In [ ]:
theta

In [ ]:
phi

In [ ]:
fft_f[0]

In [ ]:
# ROOT.Detector().GetGain_1D_OutZero(0.0,90,90,0,0)

In [ ]:
# ROOT.Detector().GetGain_1D_OutZero(300,90,phi,0,ch)

In [ ]:
detectorPtr.GetGain_1D_OutZero(1000,90,0,0,ch)

In [ ]:
theta = 90
phi = 0
pol = 0
ch = 0

gain = np.empty(fft_f.shape)
for i in range(len(gain)):
    gain[i] = detectorPtr.GetGain_1D_OutZero(fft_f[i],90,0,0,ch)
plt.plot(fft_f, gain)

In [ ]:
ch=0
theta = np.degrees(np.array(vertexReco.reco_arrivalThetas_out)[ch])
phi = np.degrees(np.array(vertexReco.reco_arrivalPhis_out)[ch])
pol = 0

gain = np.empty(fft_f.shape)
for i in range(len(gain)):
    gain[i] = detectorPtr.GetGain_1D_OutZero(fft_f[i],theta,phi,pol)
plt.plot(fft_f, gain)
plt.xlabel("Freq [MHz]")
plt.ylabel("Gain [dbi]")

In [ ]:
ch=0
theta = np.degrees(np.array(vertexReco.reco_arrivalThetas_out)[ch])
phi = np.degrees(np.array(vertexReco.reco_arrivalPhis_out)[ch])
pol = 1

gain = np.empty(fft_f.shape)
for i in range(len(gain)):
    gain[i] = detectorPtr.GetGain_1D_OutZero(fft_f[i],theta,phi,pol)
plt.plot(fft_f, gain)
plt.xlabel("Freq [MHz]")
plt.ylabel("Gain [dbi]")

In [ ]:
ch=0
theta = np.degrees(np.array(vertexReco.reco_arrivalThetas_out)[ch])
phi = np.degrees(np.array(vertexReco.reco_arrivalPhis_out)[ch])
pol_ant=0


# from pyrex.internal_functions import (normalize, complex_bilinear_interp, complex_interp)



voltage, time = util.extractChannelWaveform(usefulEvent, ch)

sampRate = len(time)/(max(time)-min(time))
b,a = signal.bessel(4, [0.15,0.4], 'bandpass', analog=False, fs=sampRate)
fft_v, fft_f, dT = util.doFFT(time,voltage)
# ff, heffs, filter_gains = getResponseAraSim(theta,phi,fft_f,pol_ant)

gains = np.empty(fft_f.shape)
heffs = np.zeros(fft_f.shape,dtype=np.complex_)
filter_gains = np.zeros(fft_f.shape,dtype=np.complex_)
# phases = []

In [ ]:
import timeit
start = timeit.default_timer()
for i in range(len(fft_f)):
    # if (i % 10 == 0):
    #     print(str(i) + '/' + str(len(fft_f)))
    gain = detectorPtr.GetGain_1D_OutZero(fft_f[i], theta, phi, pol, 0)
    heff = reportPtr.GaintoHeight(gain, fft_f[i]*1e6, 1.79)
    filter_gain = detectorPtr.GetElectGain_1D_OutZero(fft_f[i], ch)
    filter_phase = detectorPtr.GetElectPhase_1D(fft_f[i], ch)
    if(np.isnan(heff)):
        heff = 0
    phase = detectorPtr.GetAntPhase_1D(fft_f[i], theta, phi, pol)
    gains[i] = gain
    heffs[i] = heff*complex(np.cos(np.radians(phase)),np.sin(np.radians(phase)))
    filter_gains[i] = filter_gain*complex(np.cos(np.radians(filter_phase)),np.sin(np.radians(filter_phase)))
    
end = timeit.default_timer()
elapsed = end - start
print(elapsed)

In [ ]:
#Jorge's method that doesn't divide out the filter_gains
response = heffs
deDis_wf = np.divide(fft_v,abs(response))
response = np.divide(response,abs(response))
deDis_wf = np.divide(deDis_wf,response)
deDis_wf = np.nan_to_num(deDis_wf)
revert = util.doInvFFT(deDis_wf)
deDis_wf = signal.lfilter(b, a, revert)
deDis_wf = revert

In [ ]:
plt.plot(time,voltage)

In [ ]:
#Make example waveforms of IDL pulse before and after applying ant and elect factors in AraSim

In [ ]:
# plt.plot(time,voltage)
# plt.xlim([110,150])
# plt.axvline(110)

In [ ]:
plt.plot(time,deDis_wf)

In [ ]:
#Jorge's method that doesn't divide out the filter_gains
response = heffs#*filter_gains
deDis_wf = np.divide(fft_v,abs(response))
response = np.divide(response,abs(response))
deDis_wf = np.divide(deDis_wf,response)
deDis_wf = np.nan_to_num(deDis_wf)
revert = util.doInvFFT(deDis_wf)
# deDis_wf = signal.lfilter(b, a, revert)
deDis_wf = revert

plt.plot(time,deDis_wf)

In [ ]:
#Testing my math of deconvolving everything but the pol factor response.
response = (1/2)*heffs*filter_gains
deDis_wf = np.divide(fft_v,abs(response))
response = np.divide(response,abs(response))
deDis_wf1 = np.divide(deDis_wf,response)
deDis_wf2 = np.nan_to_num(deDis_wf1)
revert = util.doInvFFT(deDis_wf2)
deDis_wf3 = revert
# deDis_wf3 = signal.lfilter(b, a, revert)

plt.plot(time,deDis_wf3)

In [ ]:
#Another test of my math
ch=0
# theta = np.degrees(np.array(vertexReco.reco_arrivalThetas_out)[ch])
# phi = np.degrees(np.array(vertexReco.reco_arrivalPhis_out)[ch])

theta = np.degrees(np.array(vertexReco.true_arrivalThetas_out)[ch])
phi = np.degrees(np.array(vertexReco.true_arrivalPhis_out)[ch])


pol_ant=0
voltage, time = util.extractChannelWaveform(usefulEvent, ch)

sampRate = len(time)/(max(time)-min(time))
b,a = signal.bessel(4, [0.15,0.4], 'bandpass', analog=False, fs=sampRate)
fft_v, fft_f, dT = util.doFFT(time,voltage)

gains = np.empty(fft_f.shape)
heffs = np.zeros(fft_f.shape)
electGains = np.zeros(fft_f.shape)
phaseTerms = np.zeros(fft_f.shape,dtype=np.complex_)
antPhases = np.zeros(fft_f.shape,dtype=np.complex_)
electPhases = np.zeros(fft_f.shape,dtype=np.complex_)
initialPhases = np.zeros(fft_f.shape)
finalPhases = np.zeros(fft_f.shape)

for i in range(len(fft_f)):
    if (i % 100 == 0):
        print(str(i) + '/' + str(len(fft_f)))
    # vReal = np.real(fft_v[i])
    # vImag = np.imag(fft_v[i])
    # # initialPhase = np.arctan2(vImag,vReal) #Adding in the initial phase seems to shift the waveform in time.
    # if (vReal != 0):
    #     initialPhase = np.arctan(vImag/vReal)  
    #     if (vReal < 0):
    #         if (vImag > 0):
    #             initialPhase += np.pi
    #         elif (vImag < 0):
    #             initialPhase -= np.pi
    # else:
    #     if (vImag > 0):
    #         initialPhase = np.pi
    #     elif (vImag < 0):
    #         initialPhase = -np.pi
    #     else:
    #         initialPhase = 0
    gain = detectorPtr.GetGain_1D_OutZero(fft_f[i], theta, phi, pol, 0)
    heff = reportPtr.GaintoHeight(gain, fft_f[i], 1.79)
    antPhase = np.radians(detectorPtr.GetAntPhase_1D(fft_f[i], theta, phi, pol))
    electGain = detectorPtr.GetElectGain_1D_OutZero(fft_f[i], ch)
    electPhase = np.radians(detectorPtr.GetElectPhase_1D(fft_f[i], ch))
    # finalPhase = initialPhase + antPhase - electPhase  #Should not be subtracting out initial phase, just the phase shift.
    finalPhase = antPhase - electPhase  #Should not be subtracting out initial phase, just the phase shift.
    if(np.isnan(heff)):
        heff = 0
    gains[i] = gain
    heffs[i] = heff
    electGains[i] = electGain
    antPhases[i] = complex(np.cos(antPhase),np.sin(antPhase))
    electPhases[i] = complex(np.cos(electPhase),np.sin(electPhase))
    # initialPhases[i] = initialPhase
    finalPhases[i] = finalPhase
    # phaseTerms[i] = complex(np.cos(finalPhase),np.sin(finalPhase))

dir_res = ant.antenna.directional_response(theta=np.radians(theta), phi=np.radians(phi), polarization=polarization)(fft_f*1E6)
heffs = ant.antenna.frequency_response(fft_f*1E6)

# response = (1/2)*heffs*antPhases*electGains*electPhases
response = heffs*dir_res#*electGains*electPhases
# response = electGains*electPhases
# response = heffs*antPhases
deDis_wf = np.divide(fft_v,response)
response1 = np.divide(response,abs(response))
deDis_wf1 = np.divide(deDis_wf,response1)
deDis_wf2 = np.nan_to_num(deDis_wf1)
revert = util.doInvFFT(deDis_wf2)
deDis_wf3 = revert
deDis_wf4 = signal.lfilter(b, a, revert)

plt.plot(time,deDis_wf3)
# plt.plot(time,util.doInvFFT(np.nan_to_num(deDis_wf)))

In [ ]:
#Load IDL waveform that AraSim is sourcing
filepath = "/users/PAS0654/jflaherty13/source/AraSim/IDL1_waveformNoiseless.txt"
data = np.loadtxt(filepath, delimiter=' ')
plt.plot(data[:,0], data[:,1])
print(data.shape)

In [ ]:
filepath = "/users/PAS0654/jflaherty13/source/AraSim/IDL1_waveformNoiseless.txt"
data = np.loadtxt(filepath, delimiter=' ')
plt.plot(data[:,0], data[:,1])
plt.show()
fft, freq, dT = util.doFFT(data[:,0], data[:,1])
psd = np.abs(fft**2)
plt.plot(freq,psd)
plt.show()

In [ ]:
#Load IDL waveform that AraSim is sourcing
filepath = "/users/PAS0654/jflaherty13/source/AraSim/IDL1_waveform.txt"
data = np.loadtxt(filepath, delimiter=' ')
plt.plot(data[:,0], data[:,1])
print(data.shape)

In [ ]:
filepath = "/users/PAS0654/jflaherty13/source/AraSim/arbitrary_waveform.txt"
data = np.loadtxt(filepath, delimiter=' ')
plt.plot(data[:,0], data[:,1])
# plt.xlim([-1,10])
print(data.shape)

In [ ]:
#Interpolate the IDL waveform data to 0.25 ns between data points.
filepath = "/users/PAS0654/jflaherty13/source/AraSim/IDL1_waveformNoiseless.txt"
data = np.loadtxt(filepath, delimiter=' ')
plt.plot(data[:,0], data[:,1])
print(data.shape)

deltaT = 0.25 #ns
tMin = data[:,0].min()
tMax = data[:,0].max()

interpTmin = -50
tShift = data[0,0] - interpTmin
numDatapoints = 640
interpTmax = interpTmin + numDatapoints*deltaT

interpT = np.array(np.arange(interpTmin,interpTmax,deltaT))
interpV = np.interp(interpT, data[:,0]-tShift, data[:,1])
plt.plot(interpT,interpV)

interpData = np.empty((len(interpT),2))
interpData[:,0] = interpT
interpData[:,1] = interpV#*(1000*30)

plt.plot(interpData[:,0], interpData[:,1])
# np.savetxt("/users/PAS0654/jflaherty13/source/AraSim/IDL1_waveformNoiseless.txt", interpData)

In [ ]:
# #Want to check vertex reco thetas
# ch=0
# thetas = np.empty(vertexReco.GetEntries())
# for evt in range(len(thetas)):
#     vertexReco.GetEntry(evt)
#     thetas[evt] = np.degrees(np.array(vertexReco.reco_arrivalThetas_out)[ch])
    
# plt.plot(thetas)

In [ ]:
# ch=0
# phis = np.empty(vertexReco.GetEntries())
# for evt in range(len(thetas)):
#     vertexReco.GetEntry(evt)
#     phis[evt] = np.degrees(np.array(vertexReco.reco_arrivalPhis_out)[ch])
    
# plt.plot(phis)

In [ ]:
# ROOT.Detector(stationId=2)

In [ ]:
# freq = fft_f

# simSettingsTree = TChain("AraTree")
# simTree = TChain("AraTree2")

# for line in file_list:
#     simTree.AddFile(line)
#     simSettingsTree.AddFile(line)

# reportPtr = ROOT.Report()
# detectorPtr = ROOT.Detector()

# simTree.SetBranchAddress("report", ROOT.AddressOf(reportPtr))
# simSettingsTree.SetBranchAddress("detector", ROOT.AddressOf(detectorPtr))
# numEvents = simTree.GetEntries()


# simTree.GetEntry(0)
# simSettingsTree.GetEntry(0)

# theta = np.degrees(theta)
# phi = np.degrees(phi)
# freq = freq*1E6

# # dt = 0.3125e-9 # seconds
# # ff = np.fft.rfftfreq(int(1280/2), dt)
# gains = []
# heffs = []
# filter_gains = []
# # phases = []

In [ ]:
# for f in freq:
#     gain = detectorPtr.GetGain_1D_OutZero(f/1e6, theta, phi, pol, 0)
#     heff = reportPtr.GaintoHeight(gain, f, 1.79)
#     filter_gain = detectorPtr.GetElectGain_1D_OutZero(f/1e6)
#     filter_phase = detectorPtr.GetElectPhase_1D(f/1e6)
#     if(np.isnan(heff)):
#         heff = 0
#     phase = detectorPtr.GetAntPhase_1D(f/1e6, theta, phi, pol)
#     gains.append(gain)
#     heffs.append(heff*complex(np.cos(np.radians(phase)),np.sin(np.radians(phase))))
#     filter_gains.append(filter_gain*complex(np.cos(np.radians(filter_phase)),np.sin(np.radians(filter_phase))))

In [ ]:
# simTree.GetEntry(0)
f=400*1e6
pol=0
ch=0
theta = np.degrees(np.array(vertexReco.reco_arrivalThetas_out)[ch])
phi = np.degrees(np.array(vertexReco.reco_arrivalPhis_out)[ch])
gain = detectorPtr.GetGain_1D_OutZero(f/1e6, theta, phi, pol, 0)
heff = reportPtr.GaintoHeight(gain, f, 1.79)
# filter_gain = detectorPtr.GetElectGain_1D_OutZero(f/1e6)
# filter_phase = detectorPtr.GetElectPhase_1D(f/1e6)

print(gain)
print(heff)
# print(filter_gain)
# print(filter_phase)

In [ ]:
f = np.array(range(100,400,10))*1e6
pol=0
ch=0
theta = np.degrees(np.array(vertexReco.reco_arrivalThetas_out)[ch])
phi = np.degrees(np.array(vertexReco.reco_arrivalPhis_out)[ch])
gain = np.empty(f.shape)
for i in range(len(gain)):
    print(detectorPtr.GetGain_1D_OutZero(f[i]/1e6, theta, phi, pol, 0))

In [ ]:
print(detectorPtr.GetGain_1D(f[0]/1e6, theta, phi, 5))

In [ ]:
print(detectorPtr.GetGain_1D_OutZero(f[1]/1e6, theta, phi, pol, 0))

In [ ]:
print(detectorPtr.GetGain_1D_OutZero(f[2]/1e6, theta, phi, pol, 5))

In [ ]:
detectorPtr.GetAntPhase_1D(150,90,0,0)

In [ ]:
detectorPtr.GetAntPhase_1D(100,90,0,0)

In [ ]:
np.array(detectorPtr.freq_forfft)

In [ ]:
# gain

In [ ]:
rawFilename

In [ ]:
#Test importing deconvolvedWaveform file

# deconvolvedFilename = "/users/PAS0654/jflaherty13/source/AraSim/outputs/test/deconvolution/deconvolvedWaveforms_run_45.root"
# deconvolvedFilename = "/users/PAS0654/jflaherty13/source/AraSim/outputs/20230710_pulserSims_1000m/deconvolution/deconvolvedWaveforms_run_"+str(runNumber)+".root"

deconvolvedFile = ROOT.TFile.Open(deconvolvedFilename)

# eventTree = rawInputFile.Get("eventTree")
# simTree = rawInputFile.Get("AraTree2")

file_list = []
file_list.append(deconvolvedFilename)

eventTree2 = TChain("eventTree") #Define chain and tree that needs to be read. "VTree" in this case.
# simSettingsTree = TChain("AraTree")
# simTree = TChain("AraTree2")
# vertexReco = recoInputFile.Get("vertexReco")

for line in file_list:
    eventTree2.AddFile(line)
    # simTree.AddFile(line)
    # simSettingsTree.AddFile(line)

# reportPtr = ROOT.Report()#report pointer
eventPtr2 = ROOT.Event()
# detectorPtr = ROOT.Detector()
# interactionPtr = ROOT.Interaction()
# stationPtr = ROOT.ARA_station()

usefulEvent2 = ROOT.UsefulAtriStationEvent()
# rawEvent = ROOT.RawAtriStationEvent()
eventTree2.SetBranchAddress("UsefulAtriStationEvent",ROOT.AddressOf(usefulEvent2))
# eventTree.SetBranchAddress("RawAtriStationEvent",ROOT.AddressOf(rawEvent))
# simTree.SetBranchAddress("report",ROOT.AddressOf(reportPtr))
# simTree.SetBranchAddress("event", ROOT.AddressOf(eventPtr))
# simTree.SetBranchAddress("Nu_Interaction", ROOT.AddressOf(interactionPtr))
# simSettingsTree.SetBranchAddress("detector", ROOT.AddressOf(detectorPtr))
# simSettingsTree.SetBranchAddress("stations", ROOT.AddressOf(stationPtr))
# simSettingsTree.GetEntry(0)

totalRawEvents = eventTree2.GetEntries()
# print('total raw events:', totalRawEvents) 

# totalRecoEvents = vertexReco.GetEntries()
# print('total reco events:', totalRecoEvents)

# if (totalRecoEvents != totalRawEvents):
# print("ERROR: Raw event file and reconstructed event file have different number of events! Ensure that you are using compatible files!")
# sys.exit
print(totalRawEvents)

In [ ]:
evt=0
# vertexReco.GetEntry(evt)
eventTree2.GetEntry(evt)
# util.plotDeconvolvedWaveform(usefulEvent, vertexReco, deconvolution=False)

np.array(usefulEvent2.fVolts[2]).shape

In [ ]:
# util.plotDeconvolvedWaveform(usefulEvent2, vertexReco, deconvolution=False)

In [ ]:
ch = 0
volts = np.array(usefulEvent.fVolts[ch])
times = np.array(usefulEvent.fTimes[ch])
fig = plt.figure(figsize=(16,8))
plt.plot(times, volts)
plt.title("Raw Waveform")
plt.xlabel("Time [ns]")
plt.ylabel("Voltage [mV]")

In [ ]:
voltage, time = util.extractChannelWaveform(usefulEvent, 15)
# voltage

In [ ]:
ch = 0
volts = np.array(usefulEvent2.fVolts[ch])
times = np.array(usefulEvent2.fTimes[ch])
fig = plt.figure(figsize=(16,8))
plt.plot(times, volts)
plt.title("Deconvolved Waveform")
plt.xlabel("Time [ns]")
plt.ylabel("Voltage [mV]")

In [ ]:
fft, freq, dT = util.doFFT(times, volts)

In [ ]:
psd = np.abs(fft**2)
fig = plt.figure(figsize=(16,8))
plt.plot(freq,psd)
plt.title("Frequency Spectrum")
plt.xlabel("Frequency [MHz]")
plt.ylabel("Spectral Density [arb]")
# plt.axvline(140, color='k', linestyle='--')
# plt.axvline(825, color='k', linestyle='--')

In [ ]:
ch = 0
volts = np.array(usefulEvent.fVolts[ch])
times = np.array(usefulEvent.fTimes[ch])
volts = volts[:200]
times = times[:200]
fig = plt.figure(figsize=(16,8))
plt.plot(times, volts)
plt.title("Raw Waveform")
plt.xlabel("Time [ns]")
plt.ylabel("Voltage [mV]")

In [ ]:
ch = 0
volts = np.array(usefulEvent2.fVolts[ch])
times = np.array(usefulEvent2.fTimes[ch])
volts = volts[:200]
times = times[:200]
fig = plt.figure(figsize=(16,8))
plt.plot(times, volts)
plt.title("Deconvolved Waveform")
plt.xlabel("Time [ns]")
plt.ylabel("Voltage [mV]")

In [ ]:
fftNoise, freqNoise, dTNoise = util.doFFT(times, volts)
psdNoise = np.abs(fftNoise**2)
fig = plt.figure(figsize=(16,8))
plt.plot(freqNoise,psdNoise)
plt.title("Frequency Spectrum")
plt.xlabel("Frequency [MHz]")
plt.ylabel("Spectral Density [arb]")

In [ ]:
#Interpolate noise spectrum to match shape of signal
f = scipy.interpolate.interp1d(freqNoise, fftNoise)
fftNoiseInterp = f(freq)
psdNoiseInterp = np.abs(fftNoiseInterp**2)
fig = plt.figure(figsize=(16,8))
plt.plot(freq,psdNoiseInterp)
plt.title("Frequency Spectrum")
plt.xlabel("Frequency [MHz]")
plt.ylabel("Spectral Density [arb]")

In [ ]:
#Subtract noise from signal spectrum
fftSubtracted = fft - fftNoiseInterp
psdSubtracted = np.abs(fftSubtracted**2)
fig = plt.figure(figsize=(16,8))
plt.plot(freq,psdSubtracted)
plt.title("Frequency Spectrum")
plt.xlabel("Frequency [MHz]")
plt.ylabel("Spectral Density [arb]")

In [ ]:
ch = 0
freq = np.arange(0,1000,0.05)
gain = np.empty(freq.shape)
for i in range(len(gain)):
    gain[i] = detectorPtr.GetElectGain_1D_OutZero(freq[i],ch)
fig = plt.figure(figsize=(16,8))
plt.plot(freq, gain)
plt.title("Electronics Gain versus Frequency for Channel " + str(ch))
plt.xlabel("Frequency [MHz]")
plt.ylabel("Gain [dB?]")
plt.axvline(125, color='k', linestyle='--')
plt.axvline(850, color='k', linestyle='--')
plt.show()

In [ ]:
fig = plt.figure(figsize=(16,8))
for ch in range(16):
    freq = np.arange(0,1000,0.05)
    gain = np.empty(freq.shape)
    for i in range(len(gain)):
        gain[i] = detectorPtr.GetElectGain_1D_OutZero(freq[i],ch)
    
    plt.plot(freq, gain, label="Ch. " + str(ch))
    # plt.title("Electronics Gain versus Frequency for Channel " + str(ch))
    plt.xlabel("Frequency [MHz]")
    plt.ylabel("Gain [dB?]")


    
lowerEnd = 140
upperEnd = 825
plt.axvline(lowerEnd, color='k', linestyle='--', label=lowerEnd)
plt.axvline(upperEnd, color='k', linestyle='--', label=upperEnd)

notchFilter = 450
plt.axvline(notchFilter, color = 'k', label="Notch Filter at " + str(notchFilter))

plt.legend(fontsize=12)
plt.title("Electronics Gain versus Frequency for A2 Config 6")

In [ ]:
#Making electronics gain plot for ICRC paper showing only channels used in analysis.
fig = plt.figure(figsize=(16,8))
for ch in range(16):
    if ch in [5,7,13,15]:
        continue
    freq = np.arange(0,1000,0.05)
    gain = np.empty(freq.shape)
    for i in range(len(gain)):
        gain[i] = detectorPtr.GetElectGain_1D_OutZero(freq[i],ch)
    
    plt.plot(freq, gain, label="Ch. " + str(ch))
    # plt.title("Electronics Gain versus Frequency for Channel " + str(ch))
    plt.xlabel("Frequency [MHz]")
    plt.ylabel("Gain [dB?]")


    
# lowerEnd = 140
# upperEnd = 825
# plt.axvline(lowerEnd, color='k', linestyle='--', label=lowerEnd)
# plt.axvline(upperEnd, color='k', linestyle='--', label=upperEnd)

# notchFilter = 450
# plt.axvline(notchFilter, color = 'k', label="Notch Filter at " + str(notchFilter))

plt.legend(fontsize=12)
plt.title("Electronics Gain versus Frequency for A2 Config 6")

In [ ]:
fig = plt.figure(figsize=(16,8))
for ch in range(2):
    if (ch < 1):
        pol = 0
    else:
        pol = 1
    theta = np.degrees(np.array(vertexReco.reco_arrivalThetas_out)[ch])
    phi = 36 #np.degrees(np.array(vertexReco.reco_arrivalPhis_out)[ch])        
    freq = np.arange(0,1000,0.05)
    gain = np.empty(freq.shape)
    for i in range(len(gain)):
        gain[i] = detectorPtr.GetGain_1D_OutZero(freq[i], theta, phi, pol)
    
    plt.plot(freq, gain, label="Pol " + str(ch))
    # plt.title("Electronics Gain versus Frequency for Channel " + str(ch))
    plt.xlabel("Frequency [MHz]")
    plt.ylabel("Gain [dB?]")


    
lowerEnd = 140
upperEnd = 825
plt.axvline(lowerEnd, color='k', linestyle='--', label=lowerEnd)
plt.axvline(upperEnd, color='k', linestyle='--', label=upperEnd)

notchFilter = 450
plt.axvline(notchFilter, color = 'k', label="Notch Filter at " + str(notchFilter))

plt.legend(fontsize=12)
plt.title("Antenna Gain versus Frequency for A2 Config 6")

In [ ]:
usefulEvent2.unixTime

In [ ]:
#I need to get my deconvoluted root files working with the rest of my polReco code.
util.plotDeconvolvedWaveform(usefulEvent2, vertexReco, deconvolution=False)

In [ ]:
#I need to get my deconvoluted root files working with the rest of my polReco code.
util.plotDeconvolvedWaveform(usefulEvent2, vertexReco, deconvolution=False, channelPair=1, Hilbert=True)

In [ ]:
# usefulEvent2.getGraphFromRFChan(3)
np.array(usefulEvent.fVolts)

In [ ]:
np.array(usefulEvent.fVolts[3]).max()

In [ ]:
gr = usefulEvent2.getGraphFromRFChan(ch)
grBandPass = ROOT.FFTtools.simplePassBandFilter(gr, 125, 400)

voltageBefore = np.array(gr.GetY())
timeBefore = np.array(gr.GetX())

voltageAfter = np.array(grBandPass.GetY())
timeAfter = np.array(grBandPass.GetX())

plt.plot(timeBefore, voltageBefore)
plt.show()
plt.plot(timeAfter, voltageAfter)
plt.show()

In [ ]:

# filterType = ROOT.FFTtools.FilterTopology().LOWPASS
# butterworthFilter = ROOT.FFTtools.ButterworthFilter(filterType, 1, 400)
# # filterType.BANDPASS
# # ROOT.FFTtools.ButterworthFilter.ButterworthFilter(filterType, 1, 400)
# butterworthFilter

In [ ]:
gr.GetN()

In [ ]:
#Test building my own butterworth filter
order = 1
freqMin = 125
freqMax = 400
sampRate = freq[1]-freq[0]
sampRate

In [ ]:
# b,a = signal.bessel(4, [0.15,0.4], 'bandpass', analog=False, fs=sampRate)

In [ ]:
# plt.plot(b)

In [ ]:
# plt.plot(a)

In [ ]:
b, a = signal.butter(4, 100, 'low', analog=True)
w, h = signal.freqs(b, a)
plt.semilogx(w, 20 * np.log10(abs(h)))
plt.title('Butterworth filter frequency response')
plt.xlabel('Frequency [radians / second]')
plt.ylabel('Amplitude [dB]')
plt.margins(0, 0.1)
plt.grid(which='both', axis='both')
plt.axvline(100, color='green') # cutoff frequency
plt.show()

In [ ]:
b, a = signal.butter(4, [150,400], 'bandpass', analog=True)
w, h = signal.freqs(b, a)
plt.plot(w, (abs(h)))
plt.title('Butterworth filter frequency response')
plt.xlabel('Frequency [radians / second]')
plt.ylabel('Amplitude [dB]')
plt.margins(0, 0.1)
plt.grid(which='both', axis='both')
plt.axvline(100, color='green') # cutoff frequency
plt.show()

In [ ]:
#Write my own butterworth filter so that I can add it into my C++ script
freqMin = 150
freqMax = 400
freqs = np.arange(0,1000,0.05)
testFilter = np.ones(freqs.shape)
order = 16
for i in range(len(freqs)):
    # if (i%10 == 0):
    #     print(i)
    weight = 1
    #Play with just high and low pass functions
    weight /= np.sqrt(1 + (freqMin/freqs[i])**(2*order))
    weight /= np.sqrt(1 + (freqs[i]/freqMax)**(2*order))
    # if (freqs[i] < freqMin):
    #     weight /= 1 + (freqMin/freqs[i])**(2*order)
    # if (freqs[i] > freqMax):
    #     weight /= 1 + (freqs[i]/freqMax)**(2*order)
    testFilter[i] *= weight
plt.plot(freqs,testFilter)
plt.plot(w, (abs(h)))

In [ ]:

b, a = signal.butter(3, [150,400], 'bandpass', analog=True)
w, h = signal.freqs(b, a)
#Write my own butterworth filter so that I can add it into my C++ script
freqMin = 150
freqMax = 400
freqs = np.arange(0,1000,0.05)
testFilter = np.ones(freqs.shape)
order = 6
for i in range(len(freqs)):
    # if (i%10 == 0):
    #     print(i)
    weight = 1
    #Play with just high and low pass functions
    weight /= np.sqrt(1 + (freqMin/freqs[i])**(2*order))
    weight /= np.sqrt(1 + (freqs[i]/freqMax)**(2*order))
    # if (freqs[i] < freqMin):
    #     weight /= 1 + (freqMin/freqs[i])**(2*order)
    # if (freqs[i] > freqMax):
    #     weight /= 1 + (freqs[i]/freqMax)**(2*order)
    testFilter[i] *= weight
plt.plot(freqs,testFilter)
plt.plot(w, (abs(h)))

In [ ]:
#Write my own butterworth filter so that I can add it into my C++ script
freqMin = 150
freqMax = 400
freqs = np.arange(0,1000,0.05)
testFilter = np.ones(freqs.shape)
order = 4
for i in range(len(freqs)):
    # if (i%10 == 0):
    #     print(i)
    weight = 1
    #Play with just high and low pass functions
    weight /= np.sqrt(1 + (freqMin/freqs[i])**(4*order))
    weight /= np.sqrt(1 + (freqs[i]/freqMax)**(4*order))
    # if (freqs[i] < freqMin):
    #     weight /= 1 + (freqMin/freqs[i])**(2*order)
    # if (freqs[i] > freqMax):
    #     weight /= 1 + (freqs[i]/freqMax)**(2*order)
    testFilter[i] *= weight
fig = plt.figure(figsize=(16,8))
plt.plot(freqs,testFilter)
plt.title('Butterworth filter frequency response | Order = ' + str(order))
plt.xlabel('Frequency [MHz]')
plt.ylabel('Amplitude [arb]')
plt.margins(0, 0.1)
plt.grid(which='both', axis='both')
plt.axvline(freqMin, color='k', linestyle='--') # cutoff frequency
plt.axvline(freqMax, color='k', linestyle='--') # cutoff frequency
plt.show()

In [ ]:
#Write my own butterworth filter so that I can add it into my C++ script
freqMin = 150
freqMax = 300
freqs = np.arange(0,1000,0.05)
testFilter = np.ones(freqs.shape)
order = 8
for i in range(len(freqs)):
    # if (i%10 == 0):
    #     print(i)
    weight = 1
    #Play with just high and low pass functions
    weight /= np.sqrt(1 + (freqMin/freqs[i])**(4*order))
    weight /= np.sqrt(1 + (freqs[i]/freqMax)**(4*order))
    # if (freqs[i] < freqMin):
    #     weight /= 1 + (freqMin/freqs[i])**(2*order)
    # if (freqs[i] > freqMax):
    #     weight /= 1 + (freqs[i]/freqMax)**(2*order)
    testFilter[i] *= weight
fig = plt.figure(figsize=(16,8))
plt.plot(freqs,testFilter)
plt.title('Butterworth filter frequency response | Order = ' + str(order))
plt.xlabel('Frequency [MHz]')
plt.ylabel('Amplitude [arb]')
plt.margins(0, 0.1)
plt.grid(which='both', axis='both')
plt.axvline(freqMin, color='k', linestyle='--') # cutoff frequency
plt.axvline(freqMax, color='k', linestyle='--') # cutoff frequency
plt.show()

In [ ]:
#Butterworth filter plot for ICRC paper

#Write my own butterworth filter so that I can add it into my C++ script
freqMin = 150
freqMax = 300
freqs = np.arange(0,1000,0.05)
testFilter = np.ones(freqs.shape)
order = 8
for i in range(len(freqs)):
    # if (i%10 == 0):
    #     print(i)
    weight = 1
    #Play with just high and low pass functions
    weight /= np.sqrt(1 + (freqMin/freqs[i])**(4*order))
    weight /= np.sqrt(1 + (freqs[i]/freqMax)**(4*order))
    # if (freqs[i] < freqMin):
    #     weight /= 1 + (freqMin/freqs[i])**(2*order)
    # if (freqs[i] > freqMax):
    #     weight /= 1 + (freqs[i]/freqMax)**(2*order)
    testFilter[i] *= weight
fig = plt.figure(figsize=(16,8))
plt.plot(freqs,testFilter)
plt.title('Butterworth Filter Frequency Response')
plt.xlabel('Frequency [MHz]')
plt.ylabel('Amplitude [arb]')
plt.margins(0, 0.1)
plt.grid(which='both', axis='both')
plt.axvline(freqMin, color='k', linestyle='--') # cutoff frequency
plt.axvline(freqMax, color='k', linestyle='--') # cutoff frequency
plt.show()

In [ ]:
#Write my own butterworth filter so that I can add it into my C++ script
freqMin = 150
freqMax = 400
freqs = np.arange(0,1000,0.05)
testFilter = np.ones(freqs.shape)
order = 16
for i in range(len(freqs)):
    # if (i%10 == 0):
    #     print(i)
    weight = 1
    #Play with just high and low pass functions
    weight /= np.sqrt(1 + (freqMin/freqs[i])**(4*order))
    weight /= np.sqrt(1 + (freqs[i]/freqMax)**(4*order))
    # if (freqs[i] < freqMin):
    #     weight /= 1 + (freqMin/freqs[i])**(2*order)
    # if (freqs[i] > freqMax):
    #     weight /= 1 + (freqs[i]/freqMax)**(2*order)
    testFilter[i] *= weight
fig = plt.figure(figsize=(16,8))
plt.plot(freqs,testFilter)
plt.title('Butterworth filter frequency response | Order = ' + str(order))
plt.xlabel('Frequency [MHz]')
plt.ylabel('Amplitude [arb]')
plt.margins(0, 0.1)
plt.grid(which='both', axis='both')
plt.axvline(freqMin, color='k', linestyle='--') # cutoff frequency
plt.axvline(freqMax, color='k', linestyle='--') # cutoff frequency
plt.show()

In [ ]:
#Import spicecore data to troubleshoot graph when runngin getGraphFromRFChan()
#Test importing deconvolvedWaveform file

# deconvolvedFilename = "/users/PAS0654/jflaherty13/source/AraSim/outputs/test/deconvolution/deconvolvedWaveforms_run_45.root"
spicecoreFilename = "/users/PAS0654/jflaherty13/araAnalysis/araRecoAndSourceSearch/ARA_Reconstruction/data/A2/run_012559/split/event012559__0.root"

spicecoreFile = ROOT.TFile.Open(spicecoreFilename)

eventTree3 = spicecoreFile.Get("eventTree")

rawEvent3 = ROOT.RawAtriStationEvent()
eventTree3.SetBranchAddress("event",ROOT.AddressOf(rawEvent3))

totalRawEvents3 = eventTree3.GetEntries()
print('total raw events:', totalRawEvents3)



In [ ]:
eventTree3.GetEntry(0)
usefulEvent3 = ROOT.UsefulAtriStationEvent(rawEvent3,ROOT.AraCalType.kLatestCalib)
usefulEvent3.stationId

In [ ]:
ch=0
gr = usefulEvent3.getGraphFromRFChan(ch)
gr = ROOT.FFTtools.getInterpolatedGraph(gr,0.5)
lenGraph = gr.GetN()    
time = np.zeros(lenGraph)
voltage = np.zeros(lenGraph)
for k in range(0,gr.GetN()):
    time[k] = gr.GetX()[k]
    voltage[k] = gr.GetY()[k]
    
plt.plot(time,voltage)

In [ ]:
rawEvent3.stationId

In [ ]:
#Making example plot for ICRC paper showing different polarization angles.
primaryFont = 15
secondaryFont = 15
fig, axes = util.plotDeconvolvedWaveform(usefulEvent2, vertexReco, deconvolution=False, channelPair=1, Hilbert=True)
fig.set_size_inches(6,8)
fig.suptitle("$\Psi$ = " + str(runNumber) + "º", fontsize=2*primaryFont)
fig.supylabel("Amplitude [arb]",fontsize = primaryFont)
fig.supxlabel("Time [ns]",fontsize = primaryFont)
axes[0].set_title("VPol Channel", fontsize=secondaryFont)
axes[1].set_title("HPol Channel", fontsize=secondaryFont)

vMin=-3
vMax=3
for ax in axes:
    ax.set_ylim([vMin,vMax])

In [ ]:
#Making example plot for ICRC paper showing different polarization angles.
primaryFont = 15
secondaryFont = 15
fig, axes = util.plotDeconvolvedWaveform(usefulEvent, vertexReco, deconvolution=False, channelPair=1, Hilbert=False)
fig.set_size_inches(6,8)
fig.suptitle("$\Psi$ = " + str(runNumber) + "º", fontsize=2*primaryFont)
fig.supylabel("E-Field Amplitude [mV/m]",fontsize = primaryFont)
fig.supxlabel("Time [ns]",fontsize = primaryFont)
axes[0].set_title("VPol Channel", fontsize=secondaryFont)
axes[1].set_title("HPol Channel", fontsize=secondaryFont)

# vMin=-3
# vMax=3
# for ax in axes:
#     ax.set_ylim([vMin,vMax])